<a href="https://colab.research.google.com/github/Qu1nnD/CS290/blob/main/Shrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree
import math
from sklearn.model_selection import train_test_split

In [2]:
shrooms = pd.read_csv("https://raw.githubusercontent.com/Qu1nnD/CS290/main/data/mushroom_cleaned.csv")
shrooms.describe()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
count,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000
mean,567.257204,4.000315,2.142056,7.329509,0.759110,1051.081299,8.418062,0.952163,0.549181
std,359.883763,2.160505,2.228821,3.200266,0.650969,782.056076,3.262078,0.305594,0.497580
min,0.000000,0.000000,0.000000,0.000000,0.000426,0.000000,0.000000,0.027372,0.000000
25%,289.000000,2.000000,0.000000,5.000000,0.270997,421.000000,6.000000,0.888450,0.000000
50%,525.000000,5.000000,1.000000,8.000000,0.593295,923.000000,11.000000,0.943195,1.000000
75%,781.000000,6.000000,4.000000,10.000000,1.054858,1523.000000,11.000000,0.943195,1.000000
max,1891.000000,6.000000,6.000000,11.000000,3.835320,3569.000000,12.000000,1.804273,1.000000


In [3]:
shrooms.head()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
0,1372,2,2,10,3.807467,1545,11,1.804273,1
1,1461,2,2,10,3.807467,1557,11,1.804273,1
2,1371,2,2,10,3.612496,1566,11,1.804273,1
3,1261,6,2,10,3.787572,1566,11,1.804273,1
4,1305,6,2,10,3.711971,1464,11,0.943195,1


In [4]:
shrooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54035 entries, 0 to 54034
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cap-diameter     54035 non-null  int64  
 1   cap-shape        54035 non-null  int64  
 2   gill-attachment  54035 non-null  int64  
 3   gill-color       54035 non-null  int64  
 4   stem-height      54035 non-null  float64
 5   stem-width       54035 non-null  int64  
 6   stem-color       54035 non-null  int64  
 7   season           54035 non-null  float64
 8   class            54035 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 3.7 MB


In [5]:
target="class"
features = shrooms.columns[shrooms.columns != "class"]
features = features[features != "season"]
features

Index(['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color',
       'stem-height', 'stem-width', 'stem-color'],
      dtype='object')

In [6]:
attribute = "gill-color"

In [7]:
vals = shrooms[ attribute].unique()
vals

array([10,  5,  7,  9,  0,  3, 11,  8,  1,  6,  4,  2])

In [25]:
props = shrooms[ shrooms[attribute] == vals[11] ][target].value_counts( normalize=True )
props

pandas.core.indexes.base.Index

In [40]:
def Attribute_selection_method(dataset, target, string):
  features = dataset.columns[dataset.columns != target]
  entropys=[]
  for attribute in features:
    entropy = 0
    overall = len( dataset )
    for val in vals:
      subset_size = len(dataset[ dataset[attribute] == val ])
      weight = subset_size / overall
      props = dataset[ dataset[attribute] == val ][target].value_counts( normalize=True )
      for p in props.array:
        if(string=="entropy"):
          entropy =  entropy - weight*(p*math.log2(p))
          entropy
        else:
          entropy = entropy - weight*(p*math.log2(p))
      entropys=entropys+entropy
  minEnt=min(entropys)
  ind=entropys.indexOf(min)
  return features[ind]

In [41]:
Attribute_selection_method(shrooms,"class","entropy")

0.0004942792621750247

In [23]:
train_set, test_set = train_test_split(shrooms, test_size=0.2)

In [24]:
train_set.columns

Index(['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color',
       'stem-height', 'stem-width', 'stem-color', 'season', 'class'],
      dtype='object')

In [ ]:
train_set = train_set[["gender","age","hypertension","heart_disease","smoking_status","stroke"]]

In [ ]:
test_set = test_set[["gender","age","hypertension","heart_disease","smoking_status","stroke"]]

In [ ]:
X_train = train_set[["gender","age","hypertension","heart_disease","smoking_status"]]
y_train = train_set["stroke"]

In [ ]:
X_test = test_set[["gender","age","hypertension","heart_disease","smoking_status"]]
y_test = test_set["stroke"]

In [ ]:


num_attributes = X_train.select_dtypes( include = ['int64','float64']).columns
cat_attributes = X_train.select_dtypes( include = ['object']).columns
trf = [ ('num', StandardScaler(), num_attributes),
       ('cat', OneHotEncoder( handle_unknown='ignore'), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', DecisionTreeClassifier(max_depth=3))])
cross_val_score( pipeline, X_train, y_train, cv=5, scoring='accuracy')
model = pipeline.fit( X_train, y_train )
model
predictions = pipeline.predict( X_test )
cm = confusion_matrix( y_test, predictions )
cm
plot_tree( pipeline.named_steps['clf'])